# 0. Initializing model

Just run all the code here!

In [2]:
import pandas as pd
import torch
from scipy import sparse
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import time 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import sys
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import scipy.stats
import utils 
import BasicMF as MF
import Lensmodel as LM
import retrying
import plotly.express as px
import plotly.graph_objects as go
import plotly
import importlib
importlib.reload(LM)
#unwrapped = plotly.io._orca.request_image_with_retrying.__wrapped__
#wrapped = retrying.retry(wait_random_min=1000)(unwrapped)
#plotly.io._orca.request_image_with_retrying = wrapped

<module 'Lensmodel' from 'C:\\Users\\lamng\\projects\\BT2\\BT_Ngo\\Final\\Lensmodel.py'>

In [3]:
model = torch.load("MF_model001.pt")
lensmodel = torch.load("Lensmodel_nograd.pt")

In [4]:
#calculating knwoledge matrix; rows are students; columns are topics
stud = np.arange(118971).reshape((118971, 1))
u = model.user_factors(torch.tensor(stud))
knowledge = (torch.squeeze(lensmodel.g(u))).detach().numpy()
k = knowledge

In [5]:
data_path = "data\\train_data\\train_task_1_2.csv"
meta_task_path = "data\\metadata\\answer_metadata_task_1_2.csv"
meta_question_path = "data\\metadata\\question_metadata_task_1_2.csv"
meta_student_path = "data\\metadata\\student_metadata_task_1_2.csv"
subject_metadata_path = "data\\metadata\\subject_metadata.csv"
val_path = "data\\test_data\\test_private_answers_task_1.csv"

data = pd.read_csv(data_path)
meta_task = pd.read_csv(meta_task_path)
meta_question = pd.read_csv(meta_question_path)
meta_student = pd.read_csv(meta_student_path)
subject_metadata = pd.read_csv(subject_metadata_path)
val_data = pd.read_csv(val_path)

subject_to_idx = {subject:i for i, subject in enumerate(subject_metadata["SubjectId"])}


In [6]:
question_difficulty = data[["QuestionId", "IsCorrect"]].groupby(by=["QuestionId"]).mean()
question_difficulty.columns = ["difficulty"]
data = data.merge(question_difficulty, left_on='QuestionId', right_on='QuestionId')

def enc(vec):
    a = np.zeros(len(subject_metadata))
    ind = vec[vec.find("[")+1:vec.find("]")].split(",")
    ind = list(map(int, ind))
    ind = list(map(lambda x:subject_to_idx[x],ind))
    #print((ind))
    a[ind] = 1   
    return a
meta_question["topic"]  = meta_question["SubjectId"].apply(enc)
data = data.merge(meta_question, left_on='QuestionId', right_on='QuestionId')

In [75]:
uniqueUsers = pd.unique(data["UserId"])
uniqueQuestions= pd.unique(data["QuestionId"])
user_to_ix = {(word): (i) for i, word in enumerate(uniqueUsers)}
question_to_ix = {word: (i) for i, word in enumerate(uniqueQuestions)}
ix_to_user = { (i): (word) for i, word in enumerate(uniqueUsers)}
ix_to_question = {(i):word for i, word in enumerate(uniqueQuestions)}

# 1. High knowledge = correct answers?

Does a high value in knowledge vector lead to more correct answers?

In [266]:
share = 50
knowVector3 = []
knowVector101 = []
knowVector115 = []
knowVector342 = []
question_data = data[(data["QuestionId"] == 16997)].copy()
for index, row in question_data.iterrows():
    knowVector3.append(k[user_to_ix[row["UserId"]]][subject_to_idx[3]])
    knowVector101.append(k[user_to_ix[row["UserId"]]][subject_to_idx[101]])
    knowVector115.append(k[user_to_ix[row["UserId"]]][subject_to_idx[115]])
    knowVector342.append(k[user_to_ix[row["UserId"]]][subject_to_idx[342]])
question_data["knowledge3"] = knowVector3
question_data["knowledge101"] = knowVector101
question_data["knowledge115"] = knowVector115
question_data["knowledge342"] = knowVector342
question_data_2 = question_data[["UserId", "IsCorrect", "knowledge3", "knowledge101", "knowledge115", "knowledge342"]]
question_data_2 = question_data_2.sort_values(["knowledge3"], ascending=[True])
print(question_data_2.head(share)["IsCorrect"].mean())
print(question_data_2.tail(share)["IsCorrect"].mean())
print()
question_data_2 = question_data_2.sort_values(["knowledge101"], ascending=[True])
print(question_data_2.head(share)["IsCorrect"].mean())
print(question_data_2.tail(share)["IsCorrect"].mean())
print()
question_data_2 = question_data_2.sort_values(["knowledge115"], ascending=[True])
print(question_data_2.head(share)["IsCorrect"].mean())
print(question_data_2.tail(share)["IsCorrect"].mean())
print()
question_data_2 = question_data_2.sort_values(["knowledge342"], ascending=[True])
print(question_data_2.head(share)["IsCorrect"].mean())
print(question_data_2.tail(share)["IsCorrect"].mean())

0.76
0.8

0.8
0.76

0.82
0.88

0.78
0.9


# 2. Knowledge correlations

Calculating correlations between knowledge values

In [56]:
corr_matrix =pd.DataFrame(k).corr()
highestCorr = pd.DataFrame(np.tril(corr_matrix, k = -1)).unstack().sort_values(ascending = False)
highestCorr = highestCorr[highestCorr>0.7].to_frame()

In [57]:
#highest knowledge correlation 
overall_highest_corr = highestCorr.head(5)
for index, row in overall_highest_corr.iterrows():
    print(subject_metadata.iloc[index[0], 1])
    print(subject_metadata.iloc[index[1], 1])
    print(row.values)
    print()

Squares, Cubes, etc
Laws of Indices
[0.7923906]

Real Life Graphs
Rounding to Decimal Places
[0.78807437]

Transformations
Combined Events
[0.78002775]

Simultaneous Equations
Function Machines
[0.77813489]

Pie Chart
Distance Between Two Co-ordinates
[0.77490947]



In [58]:
#highest correlation with subject level > 1
counter = 0
for index, x in highestCorr.iterrows():
    if ((subject_metadata.iloc[index[0], 3] >1)and (subject_metadata.iloc[index[1], 3]) >1):
        print(str(index) + ": " + str(x.values))
        print(subject_metadata.iloc[index[0], 1])
        print(subject_metadata.iloc[index[1], 1])
        print()
        counter+=1
        if (counter > 9):
            break

(188, 190): [0.7923906]
Squares, Cubes, etc
Laws of Indices

(26, 157): [0.78807437]
Real Life Graphs
Rounding to Decimal Places

(221, 246): [0.78002775]
Transformations
Combined Events

(31, 297): [0.77813489]
Simultaneous Equations
Function Machines

(76, 270): [0.77490947]
Pie Chart
Distance Between Two Co-ordinates

(178, 371): [0.76197428]
Reverse Percentages (Original Amount)
Scale Drawing and Maps

(3, 178): [0.75618432]
Upper and Lower Bounds
Reverse Percentages (Original Amount)

(145, 147): [0.7558145]
Place Value
Mental Multiplication and Division

(76, 127): [0.74348813]
Pie Chart
Angle Facts with Parallel Lines

(3, 158): [0.74112118]
Upper and Lower Bounds
Rounding to Significant Figures



# 3. Tree structure analysis

Calculating correlations between parent and child subject knowledge

In [344]:
#getting all subjects with parent Geometry
child = subject_metadata[subject_metadata["ParentId"] == 71]
kmean = knowledge.mean(0)
children_knowledge = kmean[child.index].detach().numpy()
corr_matrix =pd.DataFrame(k).corr()

In [308]:
children = child.index.tolist()
child_sid = subject_metadata[subject_metadata["ParentId"] == 71]["SubjectId"].values
child_idx = subject_metadata[subject_metadata["ParentId"] == 71]["SubjectId"].index.tolist()
geom_idx = []

for c in child_sid:
    child_List=[]
    child_List.append(subject_to_idx[3])
    child_List.append(subject_to_idx[71])
    child_List.append(subject_to_idx[c])
    grandchild_ = subject_metadata[subject_metadata["ParentId"] == c]["SubjectId"].values
    for g in grandchild_:
        if g not in child_List:
            child_List.append(subject_to_idx[g])

    geom_idx.append(child_List)

In [345]:
#geom_idx contains lists of subjects; first subject is Maths, second subject is Geometry and the third is a level 2 subject; the rest is level 3 subjects
geom_idx

[[0, 39, 42, 41, 44, 46, 125, 126, 127, 128, 283, 334],
 [0, 39, 45, 129, 130, 131, 132, 335],
 [0, 39, 47, 212, 213, 336, 378],
 [0, 39, 50, 135, 136, 137, 138, 338],
 [0, 39, 58, 218, 219, 220, 339],
 [0, 39, 64, 222, 223, 224, 225, 340, 375],
 [0, 39, 65, 139, 140, 141, 142, 143, 152, 341, 385],
 [0, 39, 66, 226, 227, 342],
 [0, 39, 67, 133, 134, 343, 372, 373],
 [0, 39, 119, 51, 55, 120, 358, 386, 387],
 [0, 39, 121, 52, 54, 122, 301, 359],
 [0, 39, 123, 43, 53, 124, 299, 360],
 [0, 39, 214, 40, 56, 362],
 [0, 39, 215, 48, 57, 216, 217, 363],
 [0, 39, 221, 59, 60, 61, 62, 63, 364],
 [0, 39, 333]]

In [328]:
nr = -8
print(corr_matrix.loc[geom_idx[nr], geom_idx[nr]])

          0         39        67        133       134       343       372  \
0    1.000000 -0.626423 -0.078564 -0.440033 -0.567607  0.202640 -0.476516   
39  -0.626423  1.000000 -0.308372  0.082917  0.291485  0.056699  0.175694   
67  -0.078564 -0.308372  1.000000  0.107193 -0.361561 -0.304524 -0.019834   
133 -0.440033  0.082917  0.107193  1.000000  0.324795  0.052804  0.197701   
134 -0.567607  0.291485 -0.361561  0.324795  1.000000 -0.025643  0.467069   
343  0.202640  0.056699 -0.304524  0.052804 -0.025643  1.000000 -0.072963   
372 -0.476516  0.175694 -0.019834  0.197701  0.467069 -0.072963  1.000000   
373  0.318246  0.050125 -0.538834 -0.387912 -0.058163  0.159545 -0.079154   

          373  
0    0.318246  
39   0.050125  
67  -0.538834  
133 -0.387912  
134 -0.058163  
343  0.159545  
372 -0.079154  
373  1.000000  


In [319]:
#calcuclate correlation of Geometry and Maths
a = child_idx.copy()
print(len(child_idx))
a[:0] =  [0,subject_to_idx[71]]
print(corr_matrix.loc[a, a].iloc[0:2,1:])
print(1-(corr_matrix.loc[a, a].iloc[0,1:]>0).mean())
print(1-(corr_matrix.loc[a, a].iloc[1,1:]>0).mean())

16
         39        42        45        47        50        58        64   \
0  -0.626423 -0.334061 -0.339866 -0.369486 -0.509669 -0.054979 -0.282266   
39  1.000000 -0.110796  0.139241  0.027908  0.122082  0.078818  0.067039   

         65        66        67        119       121       123       214  \
0  -0.183924  0.071988 -0.078564  0.200818 -0.186566 -0.357090 -0.180768   
39 -0.246612 -0.173036 -0.308372 -0.287634 -0.106200 -0.129172 -0.244256   

         215       221       333  
0   0.289349 -0.409311 -0.290749  
39 -0.227925 -0.081546  0.312758  
0.8235294117647058
0.5882352941176471


# 4. Clustering

In [331]:
nrOfClusters = 9
X = k
gm = KMeans(n_clusters=nrOfClusters, random_state=0).fit(X)
cluster_labels = gm.predict(X)
uniqueUsers = pd.unique(data["UserId"])
user_to_ix = {(word): torch.tensor(i) for i, word in enumerate(uniqueUsers)}
def getCluster(user):
    s = user_to_ix[user]
    return cluster_labels[s]
    
data["cluster"] = data["UserId"].apply(getCluster)

In [334]:
modes = data[["QuestionId", "AnswerValue", "cluster"]].groupby(by=["cluster", "QuestionId"]).agg(lambda x: x.value_counts().index[0])
modes = modes.reset_index()
modes = modes.rename(columns={'AnswerValue': 'mostFrequentAnswer'})
merged = data.merge(modes, left_on = ["QuestionId", "cluster"], right_on =  ["QuestionId", "cluster"])
merged["chosenMostFrquentAnswer"] = merged["AnswerValue"] == merged["mostFrequentAnswer"]
merged2 = merged
merged2["score"] = merged2["IsCorrect"] - merged2["difficulty"]
m = merged2.groupby(by=["cluster", "SubjectId"])
m = m.agg(count=('QuestionId', 'size'), score=('score', 'mean')).reset_index()
m = m.where(m["count"]>20).dropna().sort_values("score", ascending = False)

In [340]:
for i in range(9):
    plus = m[(m["cluster"] == i) & (m["score"]>0.1)]
    minus = m[(m["cluster"] == i) & (m["score"]<-0.1)]
    scores = dict()
    counter = dict()
    for index, row in plus.iterrows():
        subjects = row["SubjectId"].replace("[", "").replace("]", "").replace(" ", "").split(",")
        #print(subjects)
        for s in subjects:
            if s.isnumeric():
                if s in scores.keys():
                    scores[s] += row["count"] * row["score"]
                    counter[s] += row["count"]
                else:
                    scores[s] = row["count"] * row["score"]    
                    counter[s] = row["count"]
    for s in scores.keys():
        scores[s] = scores[s]/counter[s]
    sc = pd.DataFrame.from_dict(scores, orient='index').sort_values(0, ascending = False)
    sc = sc.rename(columns = {0: "score"})
    sc["SubjectId"] = sc.index.astype(int)
    sc = (sc.merge(subject_metadata, left_on = "SubjectId", right_on = "SubjectId"))
    print(sc[["score", "Name"]].head(10).round(4))


    score                                    Name
0  0.1968                                Decimals
1  0.1968  Multiplying and Dividing with Decimals
2  0.1926      Right-angled Triangles (SOHCAHTOA)
3  0.1926       Graphs of Trigonometric Functions
4  0.1823                      Percentages-Others
5  0.1823                    Basic Calculator Use
6  0.1823                          Calculator Use
7  0.1770                           3D Pythagoras
8  0.1770                              Pythagoras
9  0.1770                         3D Trigonometry
    score                                              Name
0  0.1971               Writing and Simplifying Expressions
1  0.1971  Simplifying Expressions by Collecting Like Terms
2  0.1971                                 Multiplying Terms
3  0.1971                                    Dividing Terms
4  0.1639                           Second Order Derivative
5  0.1639                                     Advanced Pure
6  0.1639                     

In [341]:
for i in range(9):
    plus = m[(m["cluster"] == i) & (m["score"]>0.1)]
    minus = m[(m["cluster"] == i) & (m["score"]<-0.1)]
    scores = dict()
    counter = dict()
    for index, row in minus.iterrows():
        subjects = row["SubjectId"].replace("[", "").replace("]", "").replace(" ", "").split(",")
        #print(subjects)
        for s in subjects:
            if s.isnumeric():
                if s in scores.keys():
                    scores[s] += row["count"] * row["score"]
                    counter[s] += row["count"]
                else:
                    scores[s] = row["count"] * row["score"]    
                    counter[s] = row["count"]
    for s in scores.keys():
        scores[s] = scores[s]/counter[s]
    sc = pd.DataFrame.from_dict(scores, orient='index').sort_values(0, ascending = True)
    sc = sc.rename(columns = {0: "score"})
    sc["SubjectId"] = sc.index.astype(int)
    sc = (sc.merge(subject_metadata, left_on = "SubjectId", right_on = "SubjectId"))
    print(sc[["score", "Name"]].head(10).round(4).to_string(index=False))


  score                           Name
-0.3340                 Formula-Others
-0.3340                        Formula
-0.2285 Finding the Equation of a Line
-0.2049           Sequences and Series
-0.2049   Limiting Values of Sequences
-0.1851               Volume of Prisms
-0.1851               Area of a Circle
-0.1851        Volume and Surface Area
-0.1851                        Circles
-0.1718           Straight Line Graphs
  score                                            Name
-0.2019                                 Decimals-Others
-0.2019                                     Place Value
-0.1847                                        Decimals
-0.1813                             Rotational Symmetry
-0.1813                                   Line Symmetry
-0.1813                                        Symmetry
-0.1503                                Basic Arithmetic
-0.1470 Experimental Probability and Relative Frequency
-0.1465                      Rounding to Decimal Places
-0.1448    

In [343]:
a = merged.groupby(by=["cluster", "SubjectId"]).agg(count=('QuestionId', 'size'), agreeWithMajority=('chosenMostFrquentAnswer', 'mean')).reset_index()
a = a.sort_values("agreeWithMajority", ascending=False)
a = a.where(a["count"]>30).dropna()
for i in range(9):
    top10 = (a[a["cluster"] == i])
    #top10 = top10[top10["count"] >= 30].dropna()
    top10_head = top10.head(int(len(top10) * 0.1))
    topics = dict()
    counter = dict()
    for index, row in top10_head.iterrows():
        subjects = row["SubjectId"].replace("[", "").replace("]", "").replace(" ", "").split(",")
        #print(subjects)
        for s in subjects:
            if s.isnumeric():
                if s in topics.keys():
                    topics[s] += row["count"] * row["agreeWithMajority"]
                    counter[s] += row["count"]
                else:
                    topics[s] = row["count"] * row["agreeWithMajority"]
                    counter[s] = row["count"]
    for s in topics.keys():
        topics[s] = topics[s]/counter[s]
    agree = pd.DataFrame.from_dict(topics, orient='index').sort_values(0, ascending = False).head(10)
    agree = agree.rename(columns = {0: "agree"})
    agree["SubjectId"] = agree.index.astype(int)
    agree = (agree.merge(subject_metadata, left_on = "SubjectId", right_on = "SubjectId"))
    #agree = agree[agree["Level"] >= 2]
    print(agree[["agree", "Name"]].round(4))

    agree                                              Name
0  0.9773                            Properties of Polygons
1  0.9773                      Properties of Quadrilaterals
2  0.9773                           Properties of Triangles
3  0.9434                             Simplifying Fractions
4  0.9306                                     Number-Others
5  0.9306  Simplifying Expressions by Collecting Like Terms
6  0.9239                                    Calculator Use
7  0.9239                              Basic Calculator Use
8  0.9226                                  Written Addition
9  0.9091                                      Ratio-Others
    agree                                    Name
0  0.9524                                 Radians
1  0.9524  Converting Between Degrees and Radians
2  0.9394                         Missing Lengths
3  0.9362                   Advanced Trigonometry
4  0.9362          Basic Trigonometric Identities
5  0.9333            Properties of Quadr